# AutoGen + Assistants API DEMO
AutoGen で検証したマルチエージェントと Assistants API で開発したエージェントを接続します。

## 必要条件
- Assistants API で作成した[トラベルアシスタント](./AzureOpenAI_AssistantsAPI_Python.ipynb)
- GPT-4 Turbo with Vision モデル
- gpt-4-1106-preview モデル（Assistants API 用）*Azure OpenAI の場合
- gpt-4-0125-preview モデル（AutoGen の Agent 用）

In [ ]:
!pip install pyautogen --upgrade

In [ ]:
import autogen
print(autogen.__version__)

# 1. トラベルアシスタントの実装

`OAI_CONFIG_LIST` の設定
https://microsoft.github.io/autogen/docs/topics/llm_configuration

In [ ]:
import os

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

assistant_id = "<Your assistant_id>" #Azure OpenAI
config_list = config_list_from_json("OAI_CONFIG_LIST")


In [ ]:
config_list

## GPTAssistantAgent の作成
まだ実験的な実装ではありますが、AutoGen では Assistants API で作成したエージェントを `GPTAssistantAgent` として加えることができます。[第 1 回](./AzureOpenAI_AssistantsAPI_Python.ipynb)で開発した Assistants API の `assistant_id` を llm_config にセットします。

In [ ]:
llm_config = {"config_list": config_list, "assistant_id": assistant_id, "verbose": True}

gpt_assistant = GPTAssistantAgent(
    name="Travel Assistant", instructions="あなたはトラベルエージェントの質問応答に対応します。", llm_config=llm_config,
)

gpt_assistant

ログの機能を有効にします。以下のようにすることですべての LLM 実行ログを SQLite database に保存してくれます。

In [ ]:
logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

## スレッド ID の取得
`GPTAssistantAgent` が自動的に作成したスレッドの ID は以下のようにして取得することができます。

In [ ]:
gpt_assistant.oai_threads

## ホットペッパーAPIの実装
https://webservice.recruit.co.jp/doc/hotpepper/reference.html

In [ ]:
import requests

def search_hotpepper_shops(keyword=None, private_room=0, start=1, count=3, response_format='json'):
    base_url = "http://webservice.recruit.co.jp/hotpepper/gourmet/v1/"
    params = {
        "key": "<Your API Key>",  # 取得したAPIキーを指定
        "format": response_format,
        "start": start,
        "count": count,
        "private_room": private_room,  # 個室ありの店舗のみを検索, 0:絞り込まない, 1:絞り込む
        #"budget": "B005"
    }
    
    # キーワードが指定されている場合、パラメータに追加
    if keyword:
        params["keyword"] = keyword
        
    print("keyword:", keyword)
    response = requests.get(base_url, params=params)
    
    # レスポンス形式に応じて結果を処理
    if response_format == 'json':

        shops = []
        for s in response.json()["results"]["shop"]:
            shop = {"name": s["name"],
                    "address": s["address"],
                    "station_name": s["station_name"],
                    "access": s["access"],
                    "genre": s["genre"],
                    "budgetAverage": s["budget"]["average"],
                    "open": s["open"],
                    "close": s["close"],
                    "url": s["urls"]["pc"]
            }
            shops.append(shop)
        
        return shops#response.json()  # JSON形式のレスポンスを返す
    else:
        return response.text  # XML形式の場合は、レスポンスのテキストをそのまま返す

## 楽天トラベル空室検索APIの実装

https://webservice.rakuten.co.jp/documentation/vacant-hotel-search


In [ ]:
import requests

def search_vacant_hotels(latitude, longitude, searchRadius, checkinDate, checkoutDate, maxCharge=50000, adultNum=1, page=1, hits=2, response_format='json'):
    base_url = "https://app.rakuten.co.jp/services/api/Travel/VacantHotelSearch/20170426"
    params = {
        "applicationId": "<Your applicationId>",  # 取得した applicationId を指定
        "format": response_format,
        "page": page,
        "hits": hits,
        "latitude": latitude, # ex:35.6065914
        "longitude": longitude, # ex:139.7513225
        "searchRadius": searchRadius,  #緯度経度検索時の検索半径(単位km), 0.1 to 3.0
        "datumType": 1, # WGS84
        "checkinDate": checkinDate, # yyyy-MM-dd
        "checkoutDate": checkoutDate, # yyyy-MM-dd
        "maxCharge": maxCharge, # 上限金額, int 0 to 999999999
        "adultNum": adultNum # 宿泊者数, int 1 to 99
    }
    
    response = requests.get(base_url, params=params)
    
    # レスポンス形式に応じて結果を処理
    if response_format == 'json':
        response_data = response.json()

        if 'error' in response_data:
            error_description = response_data.get('error_description', 'No error description provided.')
            error = response_data.get('error', 'Unknown error')
            
            # エラーメッセージを処理（またはログに記録）
            print(f"Error: {error}, Description: {error_description}")
            
            # エラー情報を含むレスポンスオブジェクトまたはメッセージを返す
            return {"error": error, "error_description": error_description}
        else:
            # エラーがない場合は、通常通りレスポンスデータを返す
            hotels = []
            for hotel_group in response_data["hotels"]:
                h = hotel_group["hotel"][0]["hotelBasicInfo"]
                rooms = []
                for item in hotel_group["hotel"][1:]:
                    if "roomInfo" in item:
                        roomplans = {"roomName": item["roomInfo"][0]["roomBasicInfo"]["roomName"],
                            "stayDate": item["roomInfo"][1]["dailyCharge"]["stayDate"],
                            "rakutenCharge": item["roomInfo"][1]["dailyCharge"]["rakutenCharge"]}
                        rooms.append(roomplans)

                hotel = {"hotelNo": h["hotelNo"],
                        "hotelName": h["hotelName"],
                        "hotelSpecial": h["hotelSpecial"],
                        "access": h["access"],
                        "reviewAverage": h["reviewAverage"],
                        "hotelInformationUrl": h["hotelInformationUrl"],
                        "roomplans": rooms}
                hotels.append(hotel)
            
            return hotels
    else:
        return response.text  # XML形式の場合は、レスポンスのテキストをそのまま返す


上記のようにして、Assistants API の Tool 選択結果を実行する関数を定義し、`register_function` で登録します。これにより AutoGen が関数を実行し実行結果を Assistants API 側に返却します。

In [ ]:
gpt_assistant.register_function(
    function_map={
        "search_hotpepper_shops": search_hotpepper_shops,
        "search_vacant_hotels": search_vacant_hotels,
    }
)

# 2. 出張申請アシスタントの実装
## MultimodalConversableAgent の作成
今回は出張で使った飲食代のレシートを出張申請システムに登録するシナリオを考えます。レシートの写真をアップロードして、GPT-4 Turbo with Vision(GPT-4V) がその中の項目を適切に読み取ってから、申請システムの項目にマッピングできれば OK です。このシナリオに `ConversableAgent` クラスまで使う意味はあまりありませんが、
ただ `MultimodalConversableAgent` を使ってみたかっただけのために強引に使っています。

モデル設定のロード

In [ ]:
config_list_4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-v"],
    },
)

In [ ]:
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

image_agent = MultimodalConversableAgent(
    name="image-explainer",
    system_message = "画像を説明することのみ行うエージェントです。そのほかの質問が含まれる場合はその質問だけ無視して、画像説明のみ行います。",
    max_consecutive_auto_reply=1,
    llm_config={"config_list": config_list_4v, "temperature": 0, "max_tokens": 1000},
)

## 実際に申請処理を行うエージェント
GPT-4V によるレシート読み取り結果を、バックエンドの申請システムへ登録するエージェントです。今回あえて `MultimodalConversableAgent` を使ったため仕方なく `AssistantAgent` として実装しました。この `AssistantAgent` が Function calling によって登録関数をコールします。

In [ ]:
config_list_0125 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-0125-preview"],
    },
)

llm_config_0125 = {
    "functions": [
        {
            "name": "register_receipt_info",
            "description": "Register receipt information",
            "parameters": {
                "type": "object",
                "properties": {
                    "shopname": {
                        "type": "string",
                        "description": "the name of the shop",
                    },
                    "shopaddress": {
                        "type": "string",
                        "description": "the address of the shop",
                    },
                    "transactiondate": {
                        "type": "string",
                        "description": "the date and time of the transaction",
                    },
                    "purchaselist": {
                        "type": "string",
                        "description": 'Purchase list, JSON schema of arguments encoded as a string. For example: \'{"items": [{"name": "postcard", "price": 2000}, {"name": "candy", "price": 1500}, {"name": "beer", "price": 500, "quantity": 2}]}\' ',
                    },
                    "totalamount": {
                        "type": "string",
                        "description": "the total amount of the purchase",
                    },
                },
                "required": ["shopname", "shopaddress", "transactiondate", "purchaselist", "totalamount"],
            },
        },
    ],
    "config_list": config_list_0125
}

register_receipt_assistant = AssistantAgent(
    name="register_receipt_assistant",
    system_message="""You are an assistant who applies for receipts.
        After obtaining the receipt information, call the application function to apply.
        You may use the provided functions before providing a final answer.
        Only use the functions you were provided.
        When the answer has been provided, reply TERMINATE.""",
    llm_config=llm_config_0125,
)

`register_receipt_info` 関数はスタブです。

In [ ]:
def register_receipt_info(shopname, shopaddress, transactiondate, purchaselist, totalamount):
    print(f"Receipt information registered: {shopname}, {shopaddress}, {transactiondate}, {purchaselist}, {totalamount}")   
    return {"result": "Receipt information registered."}

register_receipt_assistant.register_function(function_map={"register_receipt_info": register_receipt_info})

# 3. マルチエージェントの構築
第 2 回で検証したマルチエージェントを `GroupChat` と `GroupChatManager` を用いて実装していきます

In [ ]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
)

生成されたコードを実行するために docker が利用できる場合は use_docker=True を設定してください。dockerを使用すると、生成されたコードを直接実行するよりも安全です。

In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, image_agent, gpt_assistant, register_receipt_assistant], messages=[], max_round=2)
groupchat

In [ ]:
config_list = config_list_from_json("OAI_CONFIG_LIST")
llm_config = {"config_list": config_list, "cache_seed": 45}
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## デフォルトのエージェント トポロジー

In [ ]:
import matplotlib.pyplot as plt  # noqa E402
import networkx as nx  # noqa E402
from autogen.graph_utils import visualize_speaker_transitions_dict  # noqa E402

agents = [user_proxy, image_agent, gpt_assistant, register_receipt_assistant]
allowed_speaker_transitions_dict = {agent: [other_agent for other_agent in agents] for agent in agents}

visualize_speaker_transitions_dict(allowed_speaker_transitions_dict, agents)

## クエリーの実行

In [ ]:
user_proxy.initiate_chat(
    manager, message="東京駅近辺でイタリアンのレストランを探しています", clear_history=True
)

In [ ]:
user_proxy.initiate_chat(
    manager, message="""このレシートを読み取ってください。次に、このレシートの情報を登録してください。
<img https://documentintelligence.ai.azure.com/documents/samples/prebuilt/receipt-japanese.jpg>."""
    , clear_history=True
)

In [ ]:
user_proxy.initiate_chat(
    manager, message="2/24に大阪に1人で出張します。大阪駅近辺の当日の居酒屋と20000円以内で泊まれる大阪駅のホテルを提案してください。", clear_history=True
)